In [4]:
## (Configuration) Allows you to return multiple variables from a single cell ##
# from IPython.core.int64eractiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = 'all'

## Allows you to import files from another folder in current directory ## 
import os 
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

#Import standard packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
import pickle
import sys

#Import metrics
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho

#Import decoder functions
from Neural_Decoding.decoders import KalmanFilterDecoder

In [5]:
# Opening Data 
folder='C:/Users/calebsponheim/OneDrive - The University of Chicago/' 

# Input
with open(folder+'_model_pickle_for_performance_comparison','rb') as f:
     sort_In=pickle.load(f,encoding='latin1') 
# Output 
with open(folder+'_model_pickle_for_performance_comparison_kin','rb') as f:
     sort_Out=pickle.load(f,encoding='latin1') 

In [6]:
# Just to see what the data looks like and to confirm dimensions
sort_In
sort_In.shape

sort_Out
sort_Out.shape

(77130, 5)

In [7]:
#Remove neurons with too few spikes in HC dataset
nd_sum=np.nansum(sort_In,axis=0) #Total number of spikes of each neuron
rmv_nrn=np.where(nd_sum<100) #Find neurons who have less than 100 spikes total
neural_data=np.delete(sort_In,rmv_nrn,1) #Remove those neurons


# # (not sure about this step)
# #Remove time bins with no output (y value)
# rmv_time=np.where(np.isnan(y_kf[:,0]) | np.isnan(y_kf[:,1]))
# X_kf=np.delete(X_kf,rmv_time,0)
# y_kf=np.delete(y_kf,rmv_time,0)

In [8]:
#The covariate is simply the matrix of firing rates for all neurons over time
X_kf=neural_data
X_kf.shape

y_kf=sort_Out
y_kf.shape

(77130, 5)

In [9]:
#Number of examples after taking into account bins removed for lag alignment
num_examples=X_kf.shape[0]

#Re-align data to take lag into account
lag=-1
if lag<0:
    y_kf=y_kf[-lag:,:]
    X_kf=X_kf[0:num_examples+lag,:]
if lag>0:
    y_kf=y_kf[0:num_examples-lag,:]
    X_kf=X_kf[lag:num_examples,:]

In [10]:
#Set what part of data should be part of the training/testing/validation sets
training_range=[0, 0.8]
valid_range=[0.8,0.9]
testing_range=[0.9, 1]

In [11]:
#Number of examples after taking into account bins removed for lag alignment
num_examples_kf=X_kf.shape[0]
        
#Note that each range has a buffer of 1 bin at the beginning and end
#This makes it so that the different sets don't include overlapping data
training_set=np.arange(np.int64(np.round(training_range[0]*num_examples_kf))+1,np.int64(np.round(training_range[1]*num_examples_kf))-1)
testing_set=np.arange(np.int64(np.round(testing_range[0]*num_examples_kf))+1,np.int64(np.round(testing_range[1]*num_examples_kf))-1)
valid_set=np.arange(np.int64(np.round(valid_range[0]*num_examples_kf))+1,np.int64(np.round(valid_range[1]*num_examples_kf))-1)

#Get training data
X_kf_train=X_kf[training_set,:]
y_kf_train=y_kf[training_set,:]

#Get testing data
X_kf_test=X_kf[testing_set,:]
y_kf_test=y_kf[testing_set,:]

#Get validation data
X_kf_valid=X_kf[valid_set,:]
y_kf_valid=y_kf[valid_set,:]

#Z-score inputs 
X_kf_train_mean=np.nanmean(X_kf_train,axis=0)
X_kf_train_std=np.nanstd(X_kf_train,axis=0)
X_kf_train=(X_kf_train-X_kf_train_mean)/X_kf_train_std
X_kf_test=(X_kf_test-X_kf_train_mean)/X_kf_train_std
X_kf_valid=(X_kf_valid-X_kf_train_mean)/X_kf_train_std

#Zero-center outputs
y_kf_train_mean=np.mean(y_kf_train,axis=0)
y_kf_train=y_kf_train-y_kf_train_mean
y_kf_test=y_kf_test-y_kf_train_mean
y_kf_valid=y_kf_valid-y_kf_train_mean

In [12]:
#Declare model
model_kf=KalmanFilterDecoder(C=1) #There is one optional parameter (see ReadMe)

#Fit model
model_kf.fit(X_kf_train,y_kf_train)

#Get predictions
y_valid_predicted_kf=model_kf.predict(X_kf_valid,y_kf_valid)

# Printing out R2s for all of the kinematics parameters 
R2_kf=get_R2(y_kf_valid,y_valid_predicted_kf)
print('R2:',R2_kf)


R2: [0.59335512 0.7129593  0.55004744 0.62235716 0.30647617]
